<a href="https://colab.research.google.com/github/Krl05oP11/SchallerPonceIA/blob/main/TFI_1_2025_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**DCDA TFI 1ra parte - Agosto 2025**

Parte 1: Importar las librerías que voy a usar.

In [2]:
# Para el manejo de datos
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Para visalización estática
import matplotlib.pyplot as plt
import seaborn as sns

# Para visualización interactiva
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Para mapas interactivos
import folium
from folium import plugins

# Para Análisis Estadístico
from scipy import stats
from scipy.stats import chi2_contingency, pearsonr

# Configuraciones de visualización
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# No mostrar los warnings
import warnings
warnings.filterwarnings('ignore')

# Aviso de tudo bôm
print("Librerías importadas OK!")
print(f"Pandas version: {pd.__version__}")
print(f"Numpy version: {np.__version__}")


Librerías importadas OK!
Pandas version: 2.2.2
Numpy version: 2.0.2


Parte 2: Cargar los datos desde los CSV de Olist

In [5]:
data_path = "/content/"  # Archivos en el directorio de Colab

# Diccionario para simplificar los nombres de archivos
archivos = {
    'orders': 'olist_orders_dataset.csv',
    'order_items': 'olist_order_items_dataset.csv',
    'order_payments': 'olist_order_payments_dataset.csv',
    'order_reviews': 'olist_order_reviews_dataset.csv',
    'products': 'olist_products_dataset.csv',
    'customers': 'olist_customers_dataset.csv',
    'geolocation': 'olist_geolocation_dataset.csv',
    'sellers': 'olist_sellers_dataset.csv',
    'product_category': 'product_category_name_translation.csv'
}

# Función para cargar los datos
# Un diccionario de DataFrames es una estructura de datos que combina dos elementos fundamentales de Python:
# Diccionarios: Estructuras de pares clave-valor ({clave: valor})
# DataFrames: Estructuras tabulares bidimensionales de la biblioteca Pandas
# ¿Qué es exactamente? Es un diccionario donde:
# 1. Las claves son identificadores (generalmente strings)
# 2. Los valores son objetos DataFrame de Pandas
def cargar_datos(archivos, data_path):
    """Carga todos los archivos CSV en un diccionario de DataFrames"""
    datasets = {}

    for nombre, archivo in archivos.items():
        try:
            ruta_completa = data_path + archivo
            datasets[nombre] = pd.read_csv(ruta_completa)
            print(f"OK! {nombre}: {datasets[nombre].shape[0]:,} filas, {datasets[nombre].shape[1]} columnas")
        except FileNotFoundError:
            print(f"):- Error: No encontré el archivo {archivo}")
        except Exception as e:
            print(f"):- Error cargando el archivo {archivo}: {str(e)}")

    return datasets

# Cargar todos los datasets desde los CSV
print(":-0 Cargando datasets...")
print("=" * 50)
data = cargar_datos(archivos, data_path)
print("=" * 50)
print(f";-) Total de datasets cargados: {len(data)}")

:-0 Cargando datasets...
OK! orders: 99,441 filas, 8 columnas
OK! order_items: 112,650 filas, 7 columnas
OK! order_payments: 103,886 filas, 5 columnas
OK! order_reviews: 99,224 filas, 7 columnas
OK! products: 32,951 filas, 9 columnas
OK! customers: 99,441 filas, 5 columnas
OK! geolocation: 1,000,163 filas, 5 columnas
OK! sellers: 3,095 filas, 4 columnas
OK! product_category: 71 filas, 2 columnas
;-) Total de datasets cargados: 9


Parte 3: Limpieza y Preparación de los datos

In [6]:
# Voy a limpiar los datos de los datasets (CSV)
# Función de alumbrado barrido y limpieza
def limpiar_datos(datasets):
    """Aplica limpieza de datos según especificaciones"""
    print("LIMPIANDO LOS DATOS")
    print("=" * 50)

    datasets_limpios = datasets.copy()

    for nombre, df in datasets_limpios.items():
        print(f"\n Limpiando a {nombre}...")

        # Identificar columnas con valores perdidos
        columnas_faltantes = df.isnull().sum()
        if columnas_faltantes.sum() > 0:
            print(f"   Encontré que hay valores perdidos:")
            for col, count in columnas_faltantes[columnas_faltantes > 0].items():
                print(f"      • {col}: {count:,} ({count/len(df)*100:.1f}%)")

                # Rellenar los perdidos según el tipo de dato. Si es número real
                # le inserto la media aritmética, si es entero le inserto la moda
                if df[col].dtype in ['float64', 'int64']:
                    promedio = df[col].mean()
                    df[col].fillna(promedio, inplace=True)
                    print(f"        → Rellenado con promedio: {promedio:.2f}")
                else:
                    moda = df[col].mode().iloc[0] if len(df[col].mode()) > 0 else 'Sin información'
                    df[col].fillna(moda, inplace=True)
                    print(f"        → Rellenado con moda: {moda}")
        else:
            print(f"   OK. No hubo valores perdidos")

        # Convertir columnas de fecha
        columnas_fecha = [col for col in df.columns if 'date' in col.lower() or 'time' in col.lower()]
        if columnas_fecha:
            print(f"   Convirtiendo fechas: {columnas_fecha}")
            for col in columnas_fecha:
                try:
                    df[col] = pd.to_datetime(df[col], errors='coerce')
                    print(f"       {col}: convertido a timestamp")
                except:
                    print(f"       {col}: error en conversión")

    print("\n OK! LIMPIEZA COMPLETADA")
    return datasets_limpios

# Aplicar limpieza
data_clean = limpiar_datos(data)

LIMPIANDO LOS DATOS

 Limpiando a orders...
   Encontré que hay valores perdidos:
      • order_approved_at: 160 (0.2%)
        → Rellenado con moda: 2018-02-27 04:31:10
      • order_delivered_carrier_date: 1,783 (1.8%)
        → Rellenado con moda: 2018-05-09 15:48:00
      • order_delivered_customer_date: 2,965 (3.0%)
        → Rellenado con moda: 2016-10-27 17:32:07
   Convirtiendo fechas: ['order_purchase_timestamp', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']
       order_purchase_timestamp: convertido a timestamp
       order_delivered_carrier_date: convertido a timestamp
       order_delivered_customer_date: convertido a timestamp
       order_estimated_delivery_date: convertido a timestamp

 Limpiando a order_items...
   OK. No hubo valores perdidos
   Convirtiendo fechas: ['shipping_limit_date']
       shipping_limit_date: convertido a timestamp

 Limpiando a order_payments...
   OK. No hubo valores perdidos

 Limpiando a 

Parte 4: Cear el Dataset Principal

In [8]:
# Super función que une las tablas más importantes
#
# CUSTOMERS (99K) ←→ ORDERS (99K) ←→ ORDER_ITEMS (113K) ←→ PRODUCTS (33K)
#                        ↓                    ↓
#                  PAYMENTS (104K)      SELLERS (3K)
#                        ↓
#                   REVIEWS (99K)
#
# GEOLOCATION (1M) ←→ ZIP_CODES (customers/sellers)
#
def crear_dataset_principal(datasets):
    """Crear dataset principal uniendo las tablas más importantes"""
    print("|:-o CREANDO DATASET PRINCIPAL")
    print("=" * 40)

    # Comenzar con orders como tabla base
    df_principal = datasets['orders'].copy()
    print(f" Base - Orders: {len(df_principal):,} registros")

    # Unir con customers
    if 'customers' in datasets:
        df_principal = df_principal.merge(
            datasets['customers'],
            on='customer_id',
            how='left'
        )
        print(f"Ok -> Customers: {len(df_principal):,} registros")

    # Unir con order_items
    if 'order_items' in datasets:
        df_principal = df_principal.merge(
            datasets['order_items'],
            on='order_id',
            how='left'
        )
        print(f"Ok -> Order Items: {len(df_principal):,} registros")

    # Unir con products
    if 'products' in datasets:
        df_principal = df_principal.merge(
            datasets['products'],
            on='product_id',
            how='left'
        )
        print(f"Ok -> Products: {len(df_principal):,} registros")

    # Unir con sellers
    if 'sellers' in datasets:
        df_principal = df_principal.merge(
            datasets['sellers'],
            on='seller_id',
            how='left'
        )
        print(f"Ok -> Sellers: {len(df_principal):,} registros")

    # Agregar información de pagos
    if 'order_payments' in datasets:
        pagos_agregados = datasets['order_payments'].groupby('order_id').agg({
            'payment_value': 'sum',
            'payment_type': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else 'unknown',
            'payment_installments': 'mean'
        }).reset_index()

        df_principal = df_principal.merge(
            pagos_agregados,
            on='order_id',
            how='left'
        )
        print(f"Ok -> Payments: {len(df_principal):,} registros")

    # Agregar información de reviews
    if 'order_reviews' in datasets:
        reviews_agregados = datasets['order_reviews'].groupby('order_id').agg({
            'review_score': 'mean'
        }).reset_index()

        df_principal = df_principal.merge(
            reviews_agregados,
            on='order_id',
            how='left'
        )
        print(f"Ok + Reviews: {len(df_principal):,} registros")

    print(f"\n ;-) Dataset principal creado: {len(df_principal):,} registros, {len(df_principal.columns)} columnas")

    return df_principal

# Crear dataset principal
df_main = crear_dataset_principal(data_clean)

# Mostrar información básica
print("\n INFORMACIÓN DEL DATASET PRINCIPAL:")
print(df_main.info())

|:-o CREANDO DATASET PRINCIPAL
 Base - Orders: 99,441 registros
Ok -> Customers: 99,441 registros
Ok -> Order Items: 113,425 registros
Ok -> Products: 113,425 registros
Ok -> Sellers: 113,425 registros
Ok -> Payments: 113,425 registros
Ok + Reviews: 113,425 registros

 ;-) Dataset principal creado: 113,425 registros, 33 columnas

 INFORMACIÓN DEL DATASET PRINCIPAL:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113425 entries, 0 to 113424
Data columns (total 33 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   order_id                       113425 non-null  object        
 1   customer_id                    113425 non-null  object        
 2   order_status                   113425 non-null  object        
 3   order_purchase_timestamp       113425 non-null  datetime64[ns]
 4   order_approved_at              113425 non-null  object        
 5   order_delivered_carrier_date   113425 no

# **>>>ANÁLISIS PARA RESPONDER LAS PREGUNTAS<<<**


---


`Pregunta 3: ¿cuáles son los productos más pedidos por estado?`

In [10]:
# Función para investigar cómo se dan los pedidos (órdenes) de compra según el estado
def analizar_distribución_estados(df):
    """Analizamos la distribución de órdenes por estado"""
    print("* ANÁLISIS: Distribución de Órdenes por Estado")
    print("=" * 50)

    if 'customer_state' in df.columns:
        distribucion_estados = df.groupby('customer_state').agg({
            'order_id': 'nunique',
            'customer_id': 'nunique',
            'payment_value': 'sum'
        }).round(2)

        distribucion_estados.columns = ['Total_Ordenes', 'Clientes_Unicos', 'Valor_Total']
        distribucion_estados['Porcentaje_Ordenes'] = (
            distribucion_estados['Total_Ordenes'] / distribucion_estados['Total_Ordenes'].sum() * 100
        ).round(2)

        distribucion_estados = distribucion_estados.sort_values('Total_Ordenes', ascending=False)
        # muestro ranking de los 10 estados top
        print("* TOP 10 ESTADOS POR CANTIDAD DE ÓRDENES:")
        print(distribucion_estados.head(10))

        print(f"\n|<- INFORMACIÓN CLAVE:")
        print(f"• El estado líder es: {distribucion_estados.index[0]} ({distribucion_estados.iloc[0]['Porcentaje_Ordenes']:.1f}% del total)")
        print(f"• Los 3 estados Top concentran: {distribucion_estados.head(3)['Porcentaje_Ordenes'].sum():.1f}% de las órdenes")
        print(f"• Total de estados activos: {len(distribucion_estados)}")

        return distribucion_estados
    else:
        print("):-() Columna 'customer_state' no encontrada")
        return None

# Ejecutar análisis
resultado_estados = analizar_distribución_estados(df_main)

* ANÁLISIS: Distribución de Órdenes por Estado
* TOP 10 ESTADOS POR CANTIDAD DE ÓRDENES:
                Total_Ordenes  Clientes_Unicos  Valor_Total  \
customer_state                                                
SP                      41746            41746   7673188.55   
RJ                      12852            12852   2783724.26   
MG                      11635            11635   2341861.47   
RS                       5466             5466   1152019.17   
PR                       5045             5045   1074614.19   
SC                       3637             3637    799135.92   
BA                       3380             3380    802416.72   
DF                       2140             2140    434512.55   
ES                       2033             2033    406946.26   
GO                       2020             2020    516182.51   

                Porcentaje_Ordenes  
customer_state                      
SP                           41.98  
RJ                           12.92  
MG    



---
`Pregunta 5: ¿cuáles son los métodos de pago más utilizados?`


---





In [12]:
# Función para investigar cuáles son los métodos de pago más usuales
def analizar_métodos_pago():
    """Analizamos los métodos de pago más utilizados"""
    print("* ANÁLISIS: Métodos de Pago Más Utilizados")
    print("=" * 50)

    if 'order_payments' in data_clean:
        df_pagos = data_clean['order_payments']

        metodos_pago = df_pagos.groupby('payment_type').agg({
            'order_id': 'count',
            'payment_value': ['sum', 'mean'],
            'payment_installments': 'mean'
        }).round(2)

        metodos_pago.columns = ['Num_Transacciones', 'Valor_Total', 'Valor_Promedio', 'Cuotas_Promedio']
        metodos_pago['Porcentaje_Uso'] = (
            metodos_pago['Num_Transacciones'] / metodos_pago['Num_Transacciones'].sum() * 100
        ).round(2)

        metodos_pago = metodos_pago.sort_values('Num_Transacciones', ascending=False)

        print(" MÉTODOS DE PAGO ORDENADOS POR USO:")
        print(metodos_pago)

        print(f"\n INFORMACIÓN CLAVE:")
        print(f"• Método más usado: {metodos_pago.index[0]} ({metodos_pago.iloc[0]['Porcentaje_Uso']:.1f}% de transacciones)")
        print(f"• Valor promedio más alto: {metodos_pago['Valor_Promedio'].idxmax()} (R$ {metodos_pago['Valor_Promedio'].max():.2f})")

        return metodos_pago
    else:
        print(")8-( Dataset de pagos no encontrado")
        return None

# Ejecutar análisis
resultado_pagos = analizar_métodos_pago()

* ANÁLISIS: Métodos de Pago Más Utilizados
 MÉTODOS DE PAGO ORDENADOS POR USO:
              Num_Transacciones  Valor_Total  Valor_Promedio  Cuotas_Promedio  \
payment_type                                                                    
credit_card               76795  12542084.19          163.32             3.51   
boleto                    19784   2869361.27          145.03             1.00   
voucher                    5775    379436.87           65.70             1.00   
debit_card                 1529    217989.79          142.57             1.00   
not_defined                   3         0.00            0.00             1.00   

              Porcentaje_Uso  
payment_type                  
credit_card            73.92  
boleto                 19.04  
voucher                 5.56  
debit_card              1.47  
not_defined             0.00  

 INFORMACIÓN CLAVE:
• Método más usado: credit_card (73.9% de transacciones)
• Valor promedio más alto: credit_card (R$ 163.32)




---
Pregunta 3:
